In [ ]:
#import library
!pip install googletrans==3.1.0a0
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from googletrans import Translator

     |████████████████████████████████| 55 kB 2.7 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 1.2 MB 18.1 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 65 kB 2.9 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=e5680e7f2cef955add60dd8aeb29e2498636c8e3ab3a2063ec8c7f9c8884e6d8
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
url = "https://www.1212.mn/tables.aspx?tbl_id=DT_NSO_1400_001V1&GH1_select_all=0&GH1SingleSelect=_2_3_4&YearM_select_all=1&YearMSingleSelect=&YearY_select_all=0&YearYSingleSelect=&viewtype=table"

In [ ]:
html_content = requests.get(url).text

In [ ]:
#get the html using Beautiful soup
soup = BeautifulSoup(html_content, "lxml")

In [ ]:
#find the table content in html
mi = soup.find_all("table", attrs={"class": "table table-bordered table-responsive table-hover"})

In [ ]:
print("Number of tables on site: ",len(mi))

Number of tables on site:  1


In [ ]:
#Extract table for the header
table1 = mi[0]

body = table1.find_all("tr")

head = body[0] 
body_rows = body[1:] 

headings = []
for item in head.find_all("th"): 
    item = (item.text).rstrip("\n")
    headings.append(item)
print(headings)

['гадаад худалдааны үндсэн үзүүлэлт', '1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06', '1997-07', '1997-08', '1997-09', '1997-10', '1997-11', '1997-12', '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06', '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12', '1999-01', '1999-02', '1999-03', '1999-04', '1999-05', '1999-06', '1999-07', '1999-08', '1999-09', '1999-10', '1999-11', '1999-12', '2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04', '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10', '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10', '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2003-10', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004

In [ ]:
#Extract the content, table row
all_rows = [] 
for row_num in range(len(body_rows)): 
    row = [] 
    for row_item in body_rows[row_num].find_all("td"): 
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        row.append(aa)
    all_rows.append(row)

In [ ]:
df = pd.DataFrame(data=all_rows,columns=headings)
df.head()

,гадаад худалдааны үндсэн үзүүлэлт,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
0,Экспорт,23.40,46.80,102.00,138.30,180.30,232.20,265.70,309.20,366.50,418.20,462.70,568.50,25.70,53.80,87.10,109.00,136.60,188.70,222.20,257.10,311.20,352.50,390.00,462.30,15.40,37.20,63.20,93.50,113.40,161.60,203.00,256.90,286.70,326.50,360.20,454.20,16.50,45.10,71.70,...,2125.40,2885.20,3581.40,4116.70,4671.20,5282.80,5944.80,6480.70,7011.80,607.26,1093.60,1771.70,2512.00,3263.40,3939.00,4571.50,5387.10,5946.10,6528.70,7055.60,7619.80,459.00,770.70,1035.80,1373.60,2006.30,2828.30,3788.70,4478.90,5147.00,5986.10,6855.10,7576.30,639.90,1334.60,2000.40,2496.70,3323.96,4111.30,4741.60
1,Импорт,15.90,35.80,70.70,103.70,126.30,180.10,220.70,265.40,303.60,343.20,404.40,468.30,25.00,54.10,95.20,146.60,186.20,230.60,262.20,299.40,337.30,383.40,433.70,503.30,14.70,44.50,82.10,114.40,137.50,193.60,239.80,272.00,308.60,338.80,372.80,512.80,20.70,54.90,96.70,...,1581.90,2178.80,2759.10,3265.70,3802.00,4351.10,4875.60,5371.50,5875.00,503.98,837.50,1285.70,1766.20,2315.00,2893.80,3507.20,4091.30,4598.80,5064.60,5583.80,6127.40,473.30,750.00,1161.60,1586.30,1992.40,2460.10,2942.00,3453.20,3928.10,4437.00,4886.60,5293.90,458.80,922.30,1523.70,1961.20,2601.60,3261.12,3838.20
2,Тэнцэл,7.50,11.00,31.30,34.60,54.10,52.10,45.00,43.80,62.80,75.00,58.30,100.30,0.70,-0.30,-8.10,-37.60,-49.60,-42.00,-40.00,-42.30,-26.10,-30.90,-43.70,-40.90,0.80,-7.30,-19.00,-20.90,-24.10,-32.00,-36.70,-15.10,-21.90,-12.40,-12.60,-58.60,-4.20,-9.80,-25.00,...,543.40,706.40,822.20,851.00,869.20,931.70,1069.20,1109.20,1136.70,103.28,256.10,485.90,745.80,948.30,1045.20,1064.30,1295.80,1347.30,1464.10,1471.80,1492.30,-14.30,20.70,-125.80,-212.70,13.90,368.20,846.60,1025.60,1218.90,1549.10,1968.50,2282.40,181.10,412.30,476.70,535.40,722.37,850.18,903.40


In [ ]:
#translate unknown words using google translate
translator = Translator()
translations = translator.translate(df.columns[0], dest='en')

In [ ]:
translations.text

'main indicators of foreign trade'

In [ ]:
#Rename header
df.rename(columns = {'гадаад худалдааны үндсэн үзүүлэлт': "Timeframe"}, inplace = True)

In [ ]:
list1 = []
for a in df["Timeframe"]:
  list1.append(translator.translate(a, dest='en').text)
list1

['Export', 'Import', 'Balance']

In [ ]:
df["Timeframe"] = pd.DataFrame({"Timeframe": list1})

In [ ]:
df

,Timeframe,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,1997-08,1997-09,1997-10,1997-11,1997-12,1998-01,1998-02,1998-03,1998-04,1998-05,1998-06,1998-07,1998-08,1998-09,1998-10,1998-11,1998-12,1999-01,1999-02,1999-03,1999-04,1999-05,1999-06,1999-07,1999-08,1999-09,1999-10,1999-11,1999-12,2000-01,2000-02,2000-03,...,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
0,Export,23.40,46.80,102.00,138.30,180.30,232.20,265.70,309.20,366.50,418.20,462.70,568.50,25.70,53.80,87.10,109.00,136.60,188.70,222.20,257.10,311.20,352.50,390.00,462.30,15.40,37.20,63.20,93.50,113.40,161.60,203.00,256.90,286.70,326.50,360.20,454.20,16.50,45.10,71.70,...,2125.40,2885.20,3581.40,4116.70,4671.20,5282.80,5944.80,6480.70,7011.80,607.26,1093.60,1771.70,2512.00,3263.40,3939.00,4571.50,5387.10,5946.10,6528.70,7055.60,7619.80,459.00,770.70,1035.80,1373.60,2006.30,2828.30,3788.70,4478.90,5147.00,5986.10,6855.10,7576.30,639.90,1334.60,2000.40,2496.70,3323.96,4111.30,4741.60
1,Import,15.90,35.80,70.70,103.70,126.30,180.10,220.70,265.40,303.60,343.20,404.40,468.30,25.00,54.10,95.20,146.60,186.20,230.60,262.20,299.40,337.30,383.40,433.70,503.30,14.70,44.50,82.10,114.40,137.50,193.60,239.80,272.00,308.60,338.80,372.80,512.80,20.70,54.90,96.70,...,1581.90,2178.80,2759.10,3265.70,3802.00,4351.10,4875.60,5371.50,5875.00,503.98,837.50,1285.70,1766.20,2315.00,2893.80,3507.20,4091.30,4598.80,5064.60,5583.80,6127.40,473.30,750.00,1161.60,1586.30,1992.40,2460.10,2942.00,3453.20,3928.10,4437.00,4886.60,5293.90,458.80,922.30,1523.70,1961.20,2601.60,3261.12,3838.20
2,Balance,7.50,11.00,31.30,34.60,54.10,52.10,45.00,43.80,62.80,75.00,58.30,100.30,0.70,-0.30,-8.10,-37.60,-49.60,-42.00,-40.00,-42.30,-26.10,-30.90,-43.70,-40.90,0.80,-7.30,-19.00,-20.90,-24.10,-32.00,-36.70,-15.10,-21.90,-12.40,-12.60,-58.60,-4.20,-9.80,-25.00,...,543.40,706.40,822.20,851.00,869.20,931.70,1069.20,1109.20,1136.70,103.28,256.10,485.90,745.80,948.30,1045.20,1064.30,1295.80,1347.30,1464.10,1471.80,1492.30,-14.30,20.70,-125.80,-212.70,13.90,368.20,846.60,1025.60,1218.90,1549.10,1968.50,2282.40,181.10,412.30,476.70,535.40,722.37,850.18,903.40


In [ ]:
#Transpose the data
df2 = df.T

In [ ]:
new_header = df2.iloc[0] 
df2 = df2[1:] 
df2.columns = new_header 

In [ ]:
df2

Timeframe,Export,Import,Balance
1997-01,23.40,15.90,7.50
1997-02,46.80,35.80,11.00
1997-03,102.00,70.70,31.30
1997-04,138.30,103.70,34.60
1997-05,180.30,126.30,54.10
...,...,...,...
2021-03,2000.40,1523.70,476.70
2021-04,2496.70,1961.20,535.40
2021-05,3323.96,2601.60,722.37
2021-06,4111.30,3261.12,850.18


In [ ]:
#Export as csv file
df2.to_csv("data.csv")

In [ ]:
url = "https://www.1212.mn/tables.aspx?tbl_id=DT_NSO_0600_009V1&A1_select_all=1&A1SingleSelect=&YearM_select_all=1&YearMSingleSelect=&viewtype=table"

In [ ]:
html_content = requests.get(url).text
soup = BeautifulSoup(html_content, "lxml")

In [ ]:
#get the html using Beautiful soup
mi = soup.find_all("table", attrs={"class": "table table-bordered table-responsive table-hover"})

In [ ]:
print("Number of tables on site: ",len(mi))

Number of tables on site:  1


In [ ]:
#Extract table for the header
table1 = mi[0]

body = table1.find_all("tr")

head = body[0] 
body_rows = body[1:] 

headings = []
for item in head.find_all("th"): 
    item = (item.text).rstrip("\n")
    headings.append(item)
print(headings)

['БҮЛЭГ', '1991-10', '1991-11', '1991-12', '1992-01', '1992-02', '1992-03', '1992-04', '1992-05', '1992-06', '1992-07', '1992-08', '1992-09', '1992-10', '1992-11', '1992-12', '1993-01', '1993-02', '1993-03', '1993-04', '1993-05', '1993-06', '1993-07', '1993-08', '1993-09', '1993-10', '1993-11', '1993-12', '1994-01', '1994-02', '1994-03', '1994-04', '1994-05', '1994-06', '1994-07', '1994-08', '1994-09', '1994-10', '1994-11', '1994-12', '1995-01', '1995-02', '1995-03', '1995-04', '1995-05', '1995-06', '1995-07', '1995-08', '1995-09', '1995-10', '1995-11', '1995-12', '1996-01', '1996-02', '1996-03', '1996-04', '1996-05', '1996-06', '1996-07', '1996-08', '1996-09', '1996-10', '1996-11', '1996-12', '1997-01', '1997-02', '1997-03', '1997-04', '1997-05', '1997-06', '1997-07', '1997-08', '1997-09', '1997-10', '1997-11', '1997-12', '1998-01', '1998-02', '1998-03', '1998-04', '1998-05', '1998-06', '1998-07', '1998-08', '1998-09', '1998-10', '1998-11', '1998-12', '1999-01', '1999-02', '1999-03', 

In [ ]:
#Extract the content, table row
all_rows = [] 
for row_num in range(len(body_rows)): 
    row = [] 
    for row_item in body_rows[row_num].find_all("td"): 
        aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
        row.append(aa)
    all_rows.append(row)

In [ ]:
df = pd.DataFrame(data=all_rows,columns=headings)
df

,БҮЛЭГ,1991-10,1991-11,1991-12,1992-01,1992-02,1992-03,1992-04,1992-05,1992-06,1992-07,1992-08,1992-09,1992-10,1992-11,1992-12,1993-01,1993-02,1993-03,1993-04,1993-05,1993-06,1993-07,1993-08,1993-09,1993-10,1993-11,1993-12,1994-01,1994-02,1994-03,1994-04,1994-05,1994-06,1994-07,1994-08,1994-09,1994-10,1994-11,1994-12,...,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07
0,Ерөнхий индекс,3.2,5.1,4.4,3.6,9.8,32.9,13.7,4.5,24.0,9.1,1.7,8.6,29.7,10.1,11.0,26.7,7.9,10.2,6.2,1.8,28.5,5.2,8.6,3.6,5.8,5.3,2.5,9.5,6.9,3.4,2.5,0.5,0.1,15.1,-1.2,3.8,7.4,3.2,2.1,...,0.9,0.6,0.6,0.5,0.5,-0.8,-0.1,1.0,1.8,0.6,0.6,0.5,0.7,1.0,1.4,0.7,-0.1,0.3,0.0,-0.3,-0.4,0.6,0.9,1.2,0.7,-0.6,0.1,0.3,0.5,-1.0,-0.3,0.4,0.6,-0.6,1.4,0.6,2.4,0.7,0.7,1.3
1,Хүнсний бараа ундаа ус,-1.8,0.9,7.1,2.1,11.5,57.5,14.3,3.9,22.7,15.4,0.8,11.8,30.8,12.0,15.7,40.2,8.4,14.0,3.9,1.7,21.1,5.4,11.9,2.4,5.1,5.9,-0.2,9.3,11.6,1.9,1.4,0.8,-2.6,16.4,-6.3,4.4,7.0,4.7,0.6,...,2.5,2.6,1.5,1.1,0.4,-4.9,-2.4,-0.1,1.8,1.8,3.4,1.0,1.9,2.8,4.2,0.6,-0.8,-1.6,-0.3,-1.5,-2.2,0.7,2.9,3.4,2.0,0.3,1.3,1.2,1.3,-4.4,-2.3,-0.2,1.5,1.5,3.9,1.4,4.3,1.8,1.1,-0.3
2,Согтууруулах ундаа тамхи,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,-0.2,0.7,0.1,0.3,0.9,-1.1,0.7,0.5,0.3,0.3,0.8,0.3,0.2,0.3,0.4,0.2,0.1,0.0,0.2,0.1,0.1,0.5,0.2,1.1,1.1,0.7,0.2,0.1,0.1,0.1,0.2,0.0,0.2,0.1,0.6,0.1,0.2,0.5,0.5,0.6
3,Хувцас бөс бараа гутал,11.9,12.4,4.9,2.5,10.2,17.6,6.3,2.5,37.8,2.1,3.4,8.2,30.6,11.8,6.1,1.9,3.2,1.4,9.1,2.9,7.8,2.3,3.5,4.6,15.3,7.0,8.9,10.3,-0.2,1.1,13.2,-1.3,9.1,-0.7,12.8,5.9,11.3,0.6,0.6,...,-0.2,-0.3,0.3,-0.4,0.5,0.8,0.7,1.3,2.1,0.5,0.7,0.4,0.6,0.7,0.5,0.7,0.4,0.4,0.3,0.6,0.6,1.0,0.3,0.3,0.4,-0.4,0.2,0.2,0.2,0.3,0.4,1.1,0.1,0.5,0.0,0.1,0.0,0.2,0.3,0.6
4,Орон сууц ус цахилгаан түлш,1.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,28.2,7.8,1.8,4.2,42.7,-0.3,2.2,2.8,0.4,12.3,0.0,0.4,215.3,0.9,0.2,17.0,0.0,0.0,0.2,-0.5,0.0,12.5,-2.1,1.2,2.0,0.5,0.2,-0.4,0.0,-0.4,32.5,...,2.0,-0.9,0.2,0.4,-0.1,0.3,1.5,2.8,4.5,1.1,-1.5,0.3,0.1,-0.7,0.2,0.3,0.1,0.3,0.4,0.3,0.1,0.3,0.4,0.2,0.0,-0.3,0.2,0.2,0.2,0.1,1.3,0.2,1.3,-10.7,1.1,0.0,12.0,-1.1,0.1,0.3
5,Гэр ахуйн тавилга гэр ахуйн бараа,0.0,10.9,0.0,15.9,23.7,17.4,24.7,3.3,29.0,5.3,-3.0,12.4,8.7,9.5,-0.6,30.5,2.0,1.3,2.2,5.2,7.6,14.9,5.9,7.1,3.3,10.7,13.0,16.8,1.7,12.1,-2.1,1.0,-2.3,0.3,9.5,5.5,0.3,1.7,2.6,...,0.2,0.5,0.3,0.3,0.2,0.2,0.2,0.4,1.0,0.3,0.2,0.3,0.3,0.3,0.4,0.4,0.2,0.3,0.5,0.3,0.1,0.8,0.2,0.4,0.3,-0.5,0.5,0.3,0.3,0.3,0.5,0.5,0.1,0.3,0.9,0.3,0.2,0.4,0.4,1.3
6,Эм тариа эмнэлгийн үйлчилгээ,0.0,0.0,0.0,0.0,0.0,0.0,96.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.1,0.0,0.0,115.4,0.0,0.0,38.7,0.0,13.2,0.0,7.5,135.1,0.0,0.0,-1.5,1.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.2,0.7,0.3,1.1,0.9,1.6,1.2,0.6,1.3,0.5,0.4,0.3,0.5,0.3,0.3,0.3,0.4,0.1,0.2,0.1,0.1,0.2,0.3,0.4,0.4,0.2,0.3,0.2,0.2,0.1,0.3,0.4,0.4,0.4,1.1,0.1,0.3,0.5,0.8,0.6
7,Тээвэр,0.0,14.7,0.0,6.6,3.0,18.2,1.5,20.8,0.0,0.0,0.0,0.0,95.1,2.6,22.4,9.1,26.5,0.0,0.0,0.0,126.7,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,80.4,0.0,0.8,3.2,0.6,1.4,...,0.2,0.0,0.7,0.1,1.5,-0.5,1.1,2.5,1.8,-0.6,-2.2,0.2,0.2,0.6,0.4,-0.3,-0.4,0.0,-0.2,-0.4,0.0,0.6,0.2,0.6,0.1,-4.5,-2.6,-0.8,0.6,0.5,-0.1,1.0,-0.1,0.0,0.5,0.3,0.1,1.0,0.9,9.0
8,Холбооны хэрэгсэл шуудангийн үйлчилгээ,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,...,-0.1,-0.1,0.0,-0.2,0.0,0.6,0.2,0.2,0.1,0.0,0.0,0.0,-0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.1,0.1
9,Амралт чөлөөт цаг соёлын бараа үйлчилгээ,14.6,0.0,0.0,1.3,2.9,11.8,52.3,9.0,4.7,5.3,2.2,-7.8,4.3,3.5,-3.3,38.9,10.3,11.1,26.7,1.4,10.7,21.8,16.6,2.5,0.6,0.6,3.9,27.2,0.1,3.3,-1.5,1.3,1.4,28.5,1.7,0.4,16.5,-0.1,1.2,...,0.4,0.1,1.1,2.5,1.5,1.2,1.3,0.5,1.1,0.3,0.2,0.1,0.2,0.6,0.4,8.9,0.0,0.1,0.1,-1.4,0.1,0.1,-0.1,0.0,0.3,-0

In [ ]:
#Google translate
translations = translator.translate(df.columns[0], dest='en')
translations.text

'CHAPTER'

In [ ]:
#Rename header
df.rename(columns = {'БҮЛЭГ': "Timeframe"}, inplace = True)

In [ ]:
list1 = []
for a in df["Timeframe"]:
  list1.append(translator.translate(a, dest='en').text)
list1

['General index',
 'Food and drink water',
 'Alcohol and smoking',
 'Clothing and footwear',
 'Apartment water electric fuel',
 'Home furniture household goods',
 'Medicines and medical services',
 'Transportation',
 'Communication facilities postal service',
 'Recreation and cultural goods and services',
 'Educational services',
 'Hotel restaurant dormitory',
 'Other goods and services']

In [ ]:
df["Timeframe"] = pd.DataFrame({"Timeframe": list1})

In [ ]:
#Transpose data
df2 = df.T

In [ ]:
new_header = df2.iloc[0] 
df2 = df2[1:] 
df2.columns = new_header 

In [ ]:
df2

Timeframe,General index,Food and drink water,Alcohol and smoking,Clothing and footwear,Apartment water electric fuel,Home furniture household goods,Medicines and medical services,Transportation,Communication facilities postal service,Recreation and cultural goods and services,Educational services,Hotel restaurant dormitory,Other goods and services
1991-10,3.2,-1.8,,11.9,1.4,0.0,0.0,0.0,,14.6,,,0.0
1991-11,5.1,0.9,,12.4,0.0,10.9,0.0,14.7,,0.0,,,3.1
1991-12,4.4,7.1,,4.9,0.0,0.0,0.0,0.0,,0.0,,,3.6
1992-01,3.6,2.1,,2.5,0.0,15.9,0.0,6.6,,1.3,,,17.7
1992-02,9.8,11.5,,10.2,0.0,23.7,0.0,3.0,,2.9,,,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03,0.6,1.4,0.1,0.1,0.0,0.3,0.1,0.3,0.1,0.0,0.4,3.7,0.6
2021-04,2.4,4.3,0.2,0.0,12.0,0.2,0.3,0.1,0.0,0.0,0.0,0.0,1.6
2021-05,0.7,1.8,0.5,0.2,-1.1,0.4,0.5,1.0,0.0,0.2,0.0,1.8,0.5
2021-06,0.7,1.1,0.5,0.3,0.1,0.4,0.8,0.9,0.1,0.1,0.0,3.5,0.3


In [ ]:
#Export as csv file
df2.to_csv("data2.csv")